In [1]:
import pandas as pd

def get_training_data(df, smol=0.01, seed=1337):
    return df.sample(frac=smol, replace=False, random_state=1337)

h1b_data = pd.read_csv('data.csv')
desired_cols = ['JOB_TITLE',
                'SOC_NAME',
                'EMPLOYER_NAME',
                'WORKSITE_STATE',
                'WORKSITE_CITY',
                'WAGE_RATE_OF_PAY_FROM']
a = h1b_data[desired_cols]
a['WAGE_RATE_OF_PAY_FROM'] = a['WAGE_RATE_OF_PAY_FROM'].apply(lambda x: int(x.replace(',','')[:-3]))
c = a[~(a['WAGE_RATE_OF_PAY_FROM'] < 10000)].dropna().copy()
TECHIES = set(['SOFTWARE', 'PROGRAMMER', 'DEVELOPER', 'ENGINEER'])

from numpy.random import rand, seed
seed(1337)

# set prior to 1 as we use all TECHIE instances for feature creation
TECHIE_PRIOR = 1
is_a_techie = lambda job_title: any([word in TECHIES for word in job_title.upper().strip().split()]) and rand() <= TECHIE_PRIOR
c['TECHIE'] = c['JOB_TITLE'].map(is_a_techie)

/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [2]:
job_freqs = c[['JOB_TITLE', 'WORKSITE_CITY']].groupby('WORKSITE_CITY').count()

# I named this variable after knowing 93 would be the number of cities with > 1000 jobs
top_93 = job_freqs[job_freqs['JOB_TITLE']>1000].sort_values('JOB_TITLE', ascending=False)
print(len(top_93))
top_93.head()

93


,JOB_TITLE
WORKSITE_CITY,
NEW YORK,29700
SAN FRANCISCO,13050
HOUSTON,11039
CHICAGO,9534
ATLANTA,9120


In [3]:
top_93['WORKSITE_CITY'] = top_93.index
print(len(c))
print(len(c[c['TECHIE']==True]))
print(len(c[c['TECHIE']==False]))
total_techies = len(c[c['TECHIE']==True])

494762
219428
275334


In [4]:
top_93_cities_series = top_93['WORKSITE_CITY']
top_93_cities = set(top_93_cities_series.tolist())

In [5]:
d = c[c['WORKSITE_CITY'].isin(top_93_cities)]

In [14]:
city_weights = d[d['TECHIE']==True][['TECHIE', 'WORKSITE_CITY']].groupby('WORKSITE_CITY').count() / total_techies
city_weights['WEIGHTS'] = city_weights.apply(lambda x: x/x.max(), axis=0)
city_weights['WORKSITE_CITY'] = city_weights.index

# to0 lazy
GOOGLE_MAPS_API_KEY = open('.env').read().split('=')[1].strip()

import requests
import json
get_lat_lng = lambda city: json.loads(requests.get('https://maps.googleapis.com/maps/api/place/textsearch/json?query={}&key={}'.format(city.replace(' ', '%20'), GOOGLE_MAPS_API_KEY)).text)['results'][0]['geometry']['location']
get_lat = lambda city: get_lat_lng(city)['lat']
get_lng = lambda city: get_lat_lng(city)['lng']
city_weights['LAT'] = city_weights['WORKSITE_CITY'].map(get_lat)
city_weights['LNG'] = city_weights['WORKSITE_CITY'].map(get_lng)
city_weights.head()

,TECHIE,WEIGHTS,WORKSITE_CITY,LAT,LNG
WORKSITE_CITY,,,,,
ALPHARETTA,0.010783,0.380937,ALPHARETTA,34.075376,-84.294090
ATLANTA,0.015107,0.533730,ATLANTA,33.748995,-84.387982
AUBURN HILLS,0.004001,0.141362,AUBURN HILLS,42.687532,-83.234103
AUSTIN,0.014146,0.499758,AUSTIN,30.267153,-97.743061
BALTIMORE,0.001782,0.062953,BALTIMORE,39.290385,-76.612189


In [7]:
weight_map = city_weights.to_dict()['TECHIE']
d['WEIGHTS'] = d['WORKSITE_CITY'].map(weight_map)
d.head()

/Users/rainierababao/anaconda2/envs/nn/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,JOB_TITLE,SOC_NAME,EMPLOYER_NAME,WORKSITE_STATE,WORKSITE_CITY,WAGE_RATE_OF_PAY_FROM,TECHIE,WEIGHTS
2,.NET SOFTWARE PROGRAMMER,COMPUTER PROGRAMMERS,EASTBANC TECHNOLOGIES LLC,DC,WASHINGTON,77000,True,0.004179
3,PROJECT MANAGER,"COMPUTER OCCUPATIONS, ALL OTHER",INFO SERVICES LLC,NJ,JERSEY CITY,102000,False,0.007018
4,ASSOCIATE - ESOTERIC ASSET BACKED SECURITIES,CREDIT ANALYSTS,BB&T CORPORATION,NY,NEW YORK,132500,False,0.028305
5,CREDIT RISK METRICS SPECIALIST,"FINANCIAL SPECIALISTS, ALL OTHER","SUNTRUST BANKS, INC.",GA,ATLANTA,71750,False,0.015107
6,BUSINESS SYSTEMS ANALYST,MANAGEMENT ANALYSTS,CITADEL INFORMATION SERVICES INC.,NJ,EDISON,61000,False,0.006645


In [17]:
del city_weights['TECHIE']
del city_weights['WORKSITE_CITY']
city_weights.to_csv('lat_lng_weights.tsv', sep='\t', index=False, header=False)